In [106]:
from parsetrees.expr_parser import webwork_parser
from parsetrees.expr_parser import Eval_parsed
import pickle
from collections import deque
from collections import defaultdict
import pandas as pd

In [107]:
data = pickle.load(open('../2015data/Week2_data.pkl','rb'))
pd.DataFrame(data[:10])

,answer,attempt,part_id,problem_id,score,set_id,timestamp,user_id,var
0,"{1,2,3,4,5,6}",{},1,1,0,Week2,2015-10-02 00:22:51,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
1,"{3,5,7,9,11}","{3,5,7,9,11}",2,1,1,Week2,2015-10-02 00:22:51,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
2,"{1,2,3,4,5,6}","{1,2,3,4,5,6}",1,1,1,Week2,2015-10-02 00:23:41,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
3,{0},{0},1,2,1,Week2,2015-10-02 00:24:52,zig006,[]
4,{1},"{0,2,3}",2,2,0,Week2,2015-10-02 00:24:52,zig006,[]
5,"{0,2,3}","{0,2,3}",3,2,1,Week2,2015-10-02 00:24:52,zig006,[]
6,{1},{1},2,2,1,Week2,2015-10-02 00:25:27,zig006,[]
7,"{4,6}",{4},1,3,0,Week2,2015-10-02 00:27:46,zig006,"[($Ac, 0), ($Bc, 0), ($B, 0), ($Cc, 0), ($C, 0..."
8,"{1,2,4,7}","{1,2,4,7}",2,3,1,Week2,2015-10-02 00:27:46,zig006,"[($Ac, 0), ($Bc, 0), ($B, 0), ($Cc, 0), ($C, 0..."
9,"{2,4,7}","{2,4,7}",3,3,1,Week2,2015-10-02 00:27:46,zig006,"[($Ac, 0), ($Bc, 0), ($B, 0), ($Cc, 0), ($C, 0..."


In [110]:
data = pickle.load(open('../2015data/Week7a_data.pkl','rb'))
pd.DataFrame(data[:10])

,answer,attempt,part_id,problem_id,score,set_id,timestamp,user_id,var
0,C,C,1,1,1,Week7a,2015-11-07 01:10:59,dtea,[]
1,A,A,1,1,1,Week7a,2015-11-07 03:22:58,quwong,[]
2,-0.02-2/[sqrt(25)]*2,yes,1,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
3,-0.02+2/[sqrt(25)]*2,yes,2,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
4,-1,yes,3,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
5,-0.02-2/[sqrt(25)]*2,1,1,5,0,Week7a,2015-11-07 03:30:37,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
6,-0.02+2/[sqrt(25)]*2,1,2,5,0,Week7a,2015-11-07 03:30:37,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
7,-1,1,3,5,0,Week7a,2015-11-07 03:30:37,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
8,-0.02-2/[sqrt(25)]*2,-1,1,5,0,Week7a,2015-11-07 03:30:44,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
9,-0.02+2/[sqrt(25)]*2,1,2,5,0,Week7a,2015-11-07 03:30:44,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."


In [49]:
def flatten(tree,tag):
    List=[]
    Queue=deque([tree])
    try:
        while Queue:
            current=Queue.popleft()
            if type(current)==list: 
                if isinstance(current[0],basestring):
                    List.append([current[1],tag,current])
                elif isinstance(current[0][0],basestring) and isinstance(current[0][1],(int, long, float, complex)) and type(current[0][2])==list:
                    List.append([current[0][1],tag,current])
                    Queue.extend(current[1:])
                else:
                    return []
            else:
                return []
    except Exception as error:
        print 'error: current=', str(current)
        return []
    List=sorted(List,key=lambda x: x[0])
    return List

In [5]:
def find_matches(params):
    attempt=params['attempt']
    attempt_tree=params['att_tree']
    attempt_list=flatten(attempt_tree,'t')
    
    answer=params['answer']
    answer_tree=params['ans_tree']
    answer_list=flatten(answer_tree,'c')

    combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])

    Hits=find_Hits(combined_list)

    final_matches=find_dominating_hits(Hits,answer,attempt)
    
    return final_matches

In [79]:
def find_Hits(List,tol = 1+1e-6):
    """ Given a combined list of subtrees from both attempt and answer,
    sorted by value, find the matching pairs of trees
    tol is the tolerance used to define which pairs of values match. 
    Needed because different expressions get different roundoff error
    """
    Hits=[]
    for i in xrange(0,len(List)-1):
        item1 = List[i]
        item2 = List[i+1]
        if item2[0] != 0:
            ratio=item1[0]/item2[0]
        else:
            ratio = 0
        if item1[1]!=item2[1] and ratio < tol and ratio > (1/tol):
            if item1[1]=='c':
                Hits.append((item1,item2))
            elif item2[1]=='c':
                Hits.append((item2,item1))
            else:
                print "Error in find_Hits. Neither item labeled c"
    return Hits

In [7]:
def find_dominating_hits(Hits,answer,attempt):
    for i in range(len(Hits)):
        for j in range(len(Hits)):
            if i==j:
                continue
            span1=get_span(Hits[i][0])
            span2=get_span(Hits[j][0])
            if span1[0]<=span2[0] and span1[1]>=span2[1]:
                #logger.debug(str(span1)+' dominates '+str(span2))
                Hits[j][0][1]='dc'
    
    final_matches=[]
    for hit in Hits:
        if hit[0][1]=='c':
            value=hit[0][0]
            span_c=get_span(hit[0])
            span_a=get_span(hit[1])
            #logger.debug('answer:'+str(answer)+str(span_c))
            #logger.debug('attempt:'+str(attempt)+str(span_a))
            answer_part=answer[span_c[0]:span_c[1]+1]
            attempt_part = attempt[span_a[0]:span_a[1]+1]
            #logger.debug('answer part'+str(answer_part))
            #logger.debug('attempt part'+str(attempt_part))
            #logger.debug('hit part='+str(hit[0][2][0]))
            if hit[0][2][0]=='X':
                node=hit[0][2][3]
            else:
                node=hit[0][2][0][3]
            final_matches.append((node,value,answer_part,attempt_part))
    return final_matches

In [68]:
def get_span(tree):
    if tree[2][0]=='X':
        return tree[2][2]
    elif type(tree[2][0])==list:
        return tree[2][0][2]
    else:
        print 'Error in get_span'
        return None

In [111]:
problem_count = defaultdict(int)
problem_part_collection = defaultdict(list)
problem_part_answer = defaultdict(str)
for d in data:
    if d['answer']:
        problem_count[(d['problem_id'], d['part_id'])] += 1
        problem_part_collection[(d['problem_id'], d['part_id'])] += [d]
        problem_part_answer[(d['problem_id'], d['part_id'])] = d['answer']
print len(data)
pd.DataFrame(problem_part_answer.items())

17828


,0,1
0,"(9, 2)",100*1000/n
1,"(2, 2)",Q(1.76777)
2,"(6, 7)","<pre style=""text-align:left; padding-left:.2em..."
3,"(5, 5)",1
4,"(7, 2)",-1
5,"(9, 5)",1.96*sqrt(100*(1000/n)*(1-1000/n))
6,"(7, 5)",1
7,"(2, 3)","<pre style=""text-align:left; padding-left:.2em..."
8,"(5, 4)",1
9,"(9, 1)",1000*100/x


In [37]:
problem_part_collection[('6','3')][0]

{'answer': u'65*(1/3*[(30/65-1/3)/(1/3)]^2+1/3*[(20/65-1/3)/(1/3)]^2+1/3*[(15/65-1/3)/(1/3)]^2)',
 'attempt': u'yes',
 u'part_id': u'3',
 u'problem_id': u'6',
 u'score': u'0',
 u'set_id': u'Week7a',
 u'timestamp': u'2015-11-07 03:35:42',
 u'user_id': u'quwong',
 'var': []}

In [41]:
params = []
count = 0
for d in problem_part_collection['6','3']:
    print count
    count += 1
    if d['answer']:
        parse_tree = webwork_parser.parse_webwork(d['answer'])
        if parse_tree == None:
            print 'parse_tree empty'
            continue
        eval_tree = Eval_parsed.eval_parsed(parse_tree[0])
        if eval_tree == None:
            print 'eval_tree empty'
            continue
        
        parse_tree_att = webwork_parser.parse_webwork(d['attempt'])
        if parse_tree_att == None:
            print 'parse_tree empty'
            continue
        eval_tree_att = Eval_parsed.eval_parsed(parse_tree_att[0])
        if eval_tree_att == None:
            print 'eval_tree empty'
            continue
            
        params += [{'user_id':d['user_id'], 'answer':d['answer'], 'attempt':d['attempt'],
                    'ans_tree':eval_tree, 'att_tree':eval_tree_att}]
    else:
        print 'no answer'

ERROR:parsetrees.expr_parser.webwork_parser:||yes|| global name 'webwork_parser' is not defined
ERROR:parsetrees.expr_parser.webwork_parser:global name 'webwork_parser' is not defined
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 649, in parseopt
    lookahead = get_token()     # Get the next token
  File "/Library/Python/2.7/site-packages/ply/lex.py", line 384, in token
    newtok = self.lexerrorf(tok)
  File "parsetrees/expr_parser/webwork_lexer.py", line 57, in t_error
    raise webwork_parser.WebworkParseException(
NameError: global name 'webwork_parser' is not defined
ERROR:parsetrees.expr_parser.webwork_parser:||60((1/3 * ((25/60 - 1/3)/(1/3))^2 + 1/3 * (

0
1
Parse Error: Unbalanced Group Operator
60((1/3 * ((25/60 - 1/3)/(1/3))^2 + 1/3 * ((20/60 - 1/3)/(1/3))^2 + 1/3 * ((15/60 - 1/3)/(1/3))^2)
  ^
2
3
4
5
6
7
8
9
10
11
12

ERROR:parsetrees.expr_parser.webwork_parser:||yes|| global name 'webwork_parser' is not defined
ERROR:parsetrees.expr_parser.webwork_parser:global name 'webwork_parser' is not defined
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 649, in parseopt
    lookahead = get_token()     # Get the next token
  File "/Library/Python/2.7/site-packages/ply/lex.py", line 384, in token
    newtok = self.lexerrorf(tok)
  File "parsetrees/expr_parser/webwork_lexer.py", line 57, in t_error
    raise webwork_parser.WebworkParseException(
NameError: global name 'webwork_parser' is not defined
ERROR:parsetrees.expr_parser.webwork_parser:||no || global name 'webwork_parser' is not d


13
14
15
16
17
18
19
20
21
22
23
24
25
26

ERROR:parsetrees.expr_parser.webwork_parser:||60((1/3)((25/60-(1/3))/(1/3))^2)+60((1/3)((20/60-(1/3))/(1/3))^2)+60((1/3)((15/60-(1/3))/(1/3))^2)|| yacc:Syntax error at 'LexToken(LPAREN,u'(',1,35)',location=35
ERROR:parsetrees.expr_parser.webwork_parser:yacc:Syntax error at 'LexToken(LPAREN,u'(',1,35)',location=35
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 792, in parseopt
    tok = self.errorfunc(errtoken)
  File "parsetrees/expr_parser/webwork_parser.py", line 304, in p_error
    raise WebworkParseException("yacc:Syntax error at '%s',location=%1d" % (p,p.lexpos))
WebworkParseException: yacc:Syntax error at 'LexToken(LPAREN,u'(',1,35)',location=35


...


595
596
597
598
Parse Error: Unbalanced Group Operator
65( (1/2(((30/65)-1/2)/1/2)^2) (1/4(((20/65)-1/4)/1/4)^2) (1/4(((20/65)-1/4)/1/4)^2)
  ^
599
600
601
602
603
604

ERROR:parsetrees.expr_parser.webwork_parser:||65*[]|| yacc:Syntax error at 'LexToken(RBRACKET,u']',1,4)',location=4
ERROR:parsetrees.expr_parser.webwork_parser:yacc:Syntax error at 'LexToken(RBRACKET,u']',1,4)',location=4
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 792, in parseopt
    tok = self.errorfunc(errtoken)
  File "parsetrees/expr_parser/webwork_parser.py", line 304, in p_error
    raise WebworkParseException("yacc:Syntax error at '%s',location=%1d" % (p,p.lexpos))
WebworkParseException: yacc:Syntax error at 'LexToken(RBRACKET,u']',1,4)',location=4
ERROR:parsetrees.expr_parser.webwork_parser:||65((1/3)((30/65-1/3)/(1/3))^2+(1/3)((20/65-1/3)/(1/3))^2+


605
606
607
608
609
610
611
612
Parse Error: Unbalanced Group Operator
65((1/3)((30/65-1/3)/(1/3))^2+(1/3)((20/65-1/3)/(1/3))^2+(1/3)((15/65-1/3)/(1/3))^2)
  ^
613
Parse Error: Unbalanced Group Operator
65*(1/3((30/65-1/3)/(1/3))^2+1/3((20/65-1/3)/(1/3))^2+1/3((15/65-1/3)/(1/3))^2)
   ^
614
615
Parse Error: Unbalanced Group Operator
65((1/3)((30/65-1/3)/(1/3))^2+(1/3)((20/65-1/3)/(1/3))^2+(1/3)((15/65-1/3)/(1/3))^2)
  ^
616
617
618
619

ERROR:parsetrees.expr_parser.webwork_parser:||65*1/3*((((30/65)-1/3)/(1/3))^2)+(((20/65)-1/3)/(1/3))^2)+(((15/65)-1/3)/(1/3))^2))|| yacc:Syntax error at 'LexToken(RPAREN,u')',1,56)',location=56
ERROR:parsetrees.expr_parser.webwork_parser:yacc:Syntax error at 'LexToken(RPAREN,u')',1,56)',location=56
Traceback (most recent call last):
  File "parsetrees/expr_parser/webwork_parser.py", line 332, in parse_webwork
    parsed = parser.parse(expr,tracking=True, lexer=lexer.lexer)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 263, in parse
    return self.parseopt(input,lexer,debug,tracking,tokenfunc)
  File "/Library/Python/2.7/site-packages/ply/yacc.py", line 792, in parseopt
    tok = self.errorfunc(errtoken)
  File "parsetrees/expr_parser/webwork_parser.py", line 304, in p_error
    raise WebworkParseException("yacc:Syntax error at '%s',location=%1d" % (p,p.lexpos))
WebworkParseException: yacc:Syntax error at 'LexToken(RPAREN,u')',1,56)',location=56



620
621


In [86]:
len(params)

608

In [87]:
clusters = defaultdict(list)
for p in params:
    final_pairs = find_matches(p)
    if len(final_pairs)>0:
        sorted_final = sorted(final_pairs,key=lambda x: x[0])
        clusters[str(sorted_final)] += [p['attempt']]
    else:
        clusters['Nothing'] += [p['attempt']]

In [99]:
for c in clusters.keys():
    if len(clusters[c]) > 10:
        print len(clusters[c]), c

17 [('R.1.0.0.1.0.0.0', 0.46153846153846156, u'30/65', u'30/65'), ('R.1.0.1.1.0.0.0', 0.3076923076923077, u'20/65', u'20/65'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0.0', 0.23076923076923078, u'15/65', u'15/65')]
220 Nothing
11 [('R.1.0.0.1.0.0.0.0', 30.0, u'30', u'30'), ('R.1.0.1.1.0.0.0.0', 20.0, u'20', u'20'), ('R.1.0.1.1.0.0.0.1', 65.0, u'65', u'65'), ('R.1.0.1.1.0.0.1', 0.3333333333333333, u'1/3', u'1/3'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0.0.0', 15.0, u'15', u'15')]
15 [('R.1.0.0.1.0.0', 0.12820512820512825, u'30/65-1/3', u'(30/65-1/3)/1'), ('R.1.0.1.1.0.0', -0.025641025641025605, u'20/65-1/3', u'(20/65-1/3)/1'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0', -0.10256410256410253, u'15/65-1/3', u'(15/65-1/3)/1')]


In [ ]:
################# TESTING BELOW ################

In [67]:
parse_tree = webwork_parser.parse_webwork('C(48,5)-3*80')
etree_a = Eval_parsed.eval_parsed(parse_tree[0])
parse_tree_b = webwork_parser.parse_webwork('C(48,5)-3*60')
etree_b = Eval_parsed.eval_parsed(parse_tree_b[0])
p = {'attempt':'C(48,5)-3*60', 'answer':'C(48,5)-3*80', 'att_tree':etree_b, 'ans_tree': etree_a}
attempt=p['attempt']
attempt_tree=p['att_tree']
attempt_list=flatten(attempt_tree,'t')

answer=p['answer']
answer_tree=p['ans_tree']
answer_list=flatten(answer_tree,'c')

combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])

Hits = find_Hits(combined_list)
Hits

[([3.0, 'c', ['X', 3.0, [8, 8], 'R.1.0']],
  [3.0, 't', ['X', 3.0, [8, 8], 'R.1.0']]),
 ([5.0, 'c', ['X', 5.0, [5, 5], 'R.0.1']],
  [5.0, 't', ['X', 5.0, [5, 5], 'R.0.1']]),
 ([48.0, 'c', ['X', 48.0, [2, 3], 'R.0.0']],
  [48.0, 't', ['X', 48.0, [2, 3], 'R.0.0']]),
 ([1712304,
   'c',
   [['C', 1712304, [0, 6], 'R.0'],
    ['X', 48.0, [2, 3], 'R.0.0'],
    ['X', 5.0, [5, 5], 'R.0.1']]],
  [1712304,
   't',
   [['C', 1712304, [0, 6], 'R.0'],
    ['X', 48.0, [2, 3], 'R.0.0'],
    ['X', 5.0, [5, 5], 'R.0.1']]])]

In [69]:
final_matches=find_dominating_hits(Hits,answer,attempt)

In [70]:
final_matches

[('R.1.0', 3.0, '3', '3'), ('R.0', 1712304, 'C(48,5)', 'C(48,5)')]